# Création automatisé du fichier Planning de Fiabilisation

Le fichier *Planning de Fiabilisation* est composé de plusieurs onglets :
* un onglet __Synthèse__ récapitule les nombres d'occurrences chargées et rejetées.
* un onglet __Calendrier__ récapitule par lot (AG01, AG02...) le nombre de rejet avec une courte analyse et un rappel du nombre de rejet du tir précédent.
* un onglet __LG11__ récapitule pour chaque occurrence rejeté le code rejet avec son libellé avec les rubriques de la table technique LG11

Nous avons mis au point ce script PYTHON parce que nous avions des problème de performance avec les fichiers XLS que nous utilisions auparavent. Les copier/coller avec le RECHERCHEV sur des dizaines de milliers de lignes mettaient notre PC à genoux. Nous proposons avec ce script une méthode plus rapide et intelligente.


## Algorithme d'analyse des rejets

Nous allons avoir en entrée 2 fichiers :

* un fichier __base de capitalisation__ qui contient tous les rejets avec les analyses correspondantes à ces rejets,
* un fichier __REJET_LG11__ qui contient dossiers rejetés

Nous allons avoir en sortie 1 fichier __Planning de Fiabilisation__ avec au moins 2 onglets :

* un onglet  __LG11__ récapitule pour chaque occurrence rejeté le code rejet avec son libellé avec les rubriques de la table technique LG11
* un onglet __Calendrier__ récapitule par lot (AG01, AG02...) le nombre de rejet avec une courte analyse.

Pour faire le lien entre un rejet capitalisé et une ligne rejeté, il faut *assembler* une clé qui est légèrement différente suivant le **Type d'erreur**. Il y a 3 types d'erreurs LG11 :

* Compatibilité : détecté par le traitement d'injection NRB (mouvements en double...)
* Dictionnaire : détecté par les traitements intrinsèques du Dictionnaire de données HRAccess.
* Utilisateur : détecté par un traitement *fonctionnel* (BNA/BNK) avec un code erreur.

#### Constitution de la clé/index de recherche des Rejets

Suivant le type de rejet, nous allons constituer une clé de recherche discriminante que nous allons implémenter pour la  __base de capitalisation__ et pour les __REJET_LG11__ .

S'il s'agit d'un type d'erreur **Compatibilité**, il faut concaténer les rubriques : *INFORMATION_HRA* + *LIB_ERREUR*

S'il s'agit d'un type d'erreur **Dictionnaire**, il faut concaténer les rubriques : *INFORMATION_HRA* + *CODE_RUBRIQUE* + *LIB_ERREUR*

S'il s'agit d'un type d'erreur **Utilisateur**, il faut concaténer les rubriques : *INFORMATION_HRA* + *CODE_ERREUR*



### Etablir la connection avec le classeur __base de capitalisation__



In [ ]:
 import os
os.environ ['PATH'] 
__version__ = '1.0.1'

In [ ]:
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys
from datetime import time, tzinfo, timedelta
class GMT1(tzinfo):
    def utcoffset(self, dt):
        return timedelta(hours=1)
    def dst(self, dt):
        return timedelta(0)
    def tzname(self,dt):
        return "Europe/Paris"
t= dt.datetime.now()
print("Début du traitement à ",t.strftime("%H:%M:%S %Z"))


Nous allons lire le fichier qui contient notre base de capitalisation. La base de capitalisation est un fichier dans lequel nous avons consigné tous les rejets que nous avons identifiés avec leur cause, ainsi qu'un moyen de corriger. Le fichier doit être dans le même répertoire que ce script.

In [ ]:
# lecture du classeur __base de capitalisation__
t= dt.datetime.now()
print("Lecture de la base de Capitalisation à ",t.strftime("%H:%M:%S %Z"))
wbBdC = xw.Book('anarejetsv1.xlsx')
# Conversion du tableau en panda.dataframe
feuilleBdC=wbBdC.sheets[0]
pdBdC = feuilleBdC.range('A1').options(pd.DataFrame,
                         index=False, expand='table').value
pdBdC.dropna()

Nous recréons un index identique entre la base de capitalisation et la LG11.

In [ ]:
# Pour chaque ligne : création de la clé
# print(pdBdC.columns) # pour identifier le nom des colonnes
t= dt.datetime.now()
print("Création de l'index des rejets de la base de capitalisation à ",t.strftime("%H:%M:%S %Z"))
for label, row in pdBdC.iterrows() :
    if row['Type_erreur'] == "Dictionnaire" :
        Cle_rejet=str(row['INFO']) + str(row ['Rubriques Hra']) + str(row['Libellé ou Code erreur'])[:32]
    if row['Type_erreur'] == "Compatibilité" or row['Type_erreur'] == "CompatibilitÃ©":
        Cle_rejet=str(row["INFO"]) + str(row['Libellé ou Code erreur'])
    if row['Type_erreur'] == "Utilisateur" :
        Cle_rejet=str(row["INFO"]) + str(row['Libellé ou Code erreur'])[:8]
    pdBdC.loc[label,"Clé Rejet"]=Cle_rejet
    
# Redéfinir le nouvel index
pdBdC.reset_index(inplace=True)
pdBdC.set_index(['Clé Rejet'], inplace=True)

# Alimentation du tableau de recherche __base de capitalisation__
# création d'un dataframe avec "Clé Rejet" et "Analyse CISIRH" uniquement pour fusion
# pdBdC_analyse=pd.DataFrame(pdBdC['Analyse CISIRH '].describe().tolist(), columns =['Analyse'])
pdBdC_analyse=pd.DataFrame({'ID Rejet': pdBdC['ID'], 'Analyse': pdBdC['Analyse CISIRH ']})
pdBdC_analyse.dropna()

A cette étape, nous allons lire l'extraction LG11 au format CSV. Le fichier doit être dans le même répertoire que ce script.
### Indiquez ici le nom du fichier LG11 à analyser

In [ ]:
# Lecture du classeur __REJET_LG11__
wbLG11 = xw.Book('REJET_AGENT_LOT1_MAA.csv')

In [ ]:
# On commence à partir de la ligne d'entête qui commence avec "TYPE_DOSSIER"
feuilleLG11=wbLG11.sheets[0]
for i,cellule in enumerate(feuilleLG11.range('A1:A95')):
    if cellule.value == "TYPE_DOSSIER" :
        debutLG11 = cellule.address
        print ("Début de la LG11 :", debutLG11)

In [ ]:
# Conversion du tableau en panda.dataframe
feuilleLG11=wbLG11.sheets[0]
pdLG11 = feuilleLG11.range(debutLG11).options(pd.DataFrame,
                         index=False, expand='table').value
# pdLG11.dropna() # pour supprimer les lignes vides

# print(pdLG11.columns) # pour debug afficher les colonnes du fichier

In [ ]:
t= dt.datetime.now()
print ("Création de l'index des rejets de la LG11 à ",t.strftime("%H:%M:%S %Z"))
# Pour chaque ligne : création de la clé
for label, row in pdLG11.iterrows() :
    if row['GRAVITE'] == "Erreur bloquante":
        if row['TYPE_ERREUR'] == "Dictionnaire" :
            Cle_rejet=str(row['INFORMATION_HRA']) + str(row ['CODE_RUBRIQUE']) + str(row['LIB_ERREUR'])[:32]
        if row['TYPE_ERREUR'] == "Compatibilité" or row['TYPE_ERREUR'] == "CompatibilitÃ©" :
            Cle_rejet=str(row['INFORMATION_HRA']) + str(row['LIB_ERREUR'])
        if row['TYPE_ERREUR'] == "Utilisateur" :
            Cle_rejet=str(row['INFORMATION_HRA']) + str(row['CODE_ERREUR'])[:8]
        pdLG11.loc[label,"Clé Rejet"]=Cle_rejet
  
# Redéfinir le nouvel index
pdLG11.reset_index(inplace=True)
try: 
    pdLG11.set_index(['Clé Rejet'], inplace=True)
except KeyError:
    print("Pas de rejets trouvés !")
# pdLG11.dropna()


### Lister les rejets non documentés

__ATTENTION__ : Il faut vérifier que toutes les clés de rejets de la LG11 existe dans la Base de Capitalisation.
On créé un dataframe temporaire (pdLG11_analyse) pour stocker les codes rejets et détecter plus facilement les codes rejets absents de la base de capitalisation.

In [ ]:
pdLG11_analyse=pd.DataFrame(pdLG11.iloc[:,3])
# print(pdLG11_analyse) # pour debug

# le tilde permet de NE PAS selectionner toutes les clés de rejet de pdLG11 qui sont dans pdBdC
Rejets_non_trouves = pdLG11_analyse[~pdLG11_analyse.index.isin(pdBdC_analyse.index)]
# print(Rejets_non_trouves) # pour debug
# On supprime les doublons
Rejets_uniques_NA = Rejets_non_trouves.index.drop_duplicates().dropna()
# print(Rejets_uniques_NA) # pour debug
# !!! LISTE DES REJETS A AMELIORER !!!
liste_rejets_NA=list(Rejets_uniques_NA)
# Si la liste contient au moins 1 rejet : on liste
for rejet in liste_rejets_NA :
    print("Code non trouvé ",rejet)

### Fusion de la table de capitalisation avec la table LG11 pour obtenir le *planning de fiabilisation* ###

Après avoir stocké dans des tables Panda nos données, nous allons fusionner ces 2 tables sur la Clé Rejet avec *pd.merge*.
Nous allons créé un fichier Excel vide dans lequel nous allons mettre le résultat.

In [ ]:
t= dt.datetime.now()
print("Création du Planning de fiabilisation à ",t.strftime("%H:%M:%S %Z"))
# recherche dans tableau de recherche __base de capitalisation__
# avec le mécanisme de fusion Panda (pd.merge) du tableau LG11 et du tableau Base de connaissances
# Voir pour la création d'un nom de Classeur à partir du Lot LG11
print ("Création du classeur ")
wbPdF = xw.Book()  # Ceci va créer un nouveau classeur Planning de Fiabilisation
# création de la feuille LG11
feuillePdF = wbPdF.sheets[0]

# Alimentation de l'entête du fichier LG11
feuillePdF.range('A1').value = ['ID','Analyse_CISIRH','TYPE_DOSSIER','IDENTIFIANT_DOSSIER_HRA','ANCIEN_IDENTIFIANT','ID1_OCCURRENCE','ID2_OCCURRENCE','ID3_OCCURRENCE','ID4_OCCURRENCE','ID5_OCCURRENCE','ID6_OCCURRENCE','DATE_REJET','HORODATAGE_ERREUR','INFORMATION_HRA','TNAME','CODE_GRAVITE','GRAVITE','CODE_RUBRIQUE','CNAME','VALEUR','TYPE_ERREUR','LIB_ERREUR','CODE_ERREUR','CODE_TRAITEMENT','REPERTOIRE_HRA','NOMENCLATURE','ID_TIR','zontri'
]
# Cela nous permet de récupérer l'instance spécifique d'Excel avec son pid
pid = xw.apps.keys()[0] # ou vous pouvez utiliser xw.apps.active.pid
try:
    planning_fiab1=pd.merge(pdLG11, pdBdC_analyse,on='Clé Rejet',how='left')
except KeyError:
    print("Pas de tableau à générer !")

# Ecriture dans le fichier Planning de Fiabilisation Onglet LG11

feuillePdF.range('A1').options(index=False).value = pd.DataFrame(planning_fiab1, columns=['ID Rejet', 'Analyse','TYPE_DOSSIER', 'IDENTIFIANT_DOSSIER_HRA',
       'ANCIEN_IDENTIFIANT', 'ID1_OCCURRENCE', 'ID2_OCCURRENCE',
       'ID3_OCCURRENCE', 'ID4_OCCURRENCE', 'ID5_OCCURRENCE', 'ID6_OCCURRENCE',
       'DATE_REJET', 'HORODATAGE_ERREUR', 'INFORMATION_HRA', 'TNAME',
       'CODE_GRAVITE', 'GRAVITE', 'CODE_RUBRIQUE', 'CNAME', 'VALEUR',
       'TYPE_ERREUR', 'LIB_ERREUR', 'CODE_ERREUR', 'CODE_TRAITEMENT',
       'REPERTOIRE_HRA', 'NOMENCLATURE', 'ID_TIR', 'zontri'])
             


In [ ]:
# A la fin de la lecture du classeur __REJET_LG11__ : création de l'onglet __Calendrier__
# Création de la feuille de statistique des rejets
feuilleStatPdF = wbPdF.sheets.add(name='Stats', after='Feuil1')
# 
t= dt.datetime.now()
print("Création du Compte-rendu des rejets à ",t.strftime("%H:%M:%S %Z"))

In [ ]:
# Comptage du nombre de rejets en groupant par ID Rejet & Analyse
statrejets = planning_fiab1.groupby(['ID Rejet', 'Analyse'], as_index=False).count()
# Ecriture dans l'onglet Stats de 3 colonnes uniquement 'ID Rejet', 'Analyse', 'INFORMATION_HRA'
feuilleStatPdF.range('A1').options(index=False).value = pd.DataFrame(statrejets,  columns=['ID Rejet', 'Analyse', 'INFORMATION_HRA'])

# Petite mise en page avec
# autofit des colonnes à partir d'une plage
feuilleStatPdF.range('A:C').columns.autofit()
# Assigne un tuple RGB Gris
feuilleStatPdF.range('A1:C1').color = (216, 216, 216)
#
t= dt.datetime.now()
print("Terminé à ",t.strftime("%H:%M:%S %Z"))

Il ne reste plus qu'à Copier/Coller le résultat dans le fichier consolidé.